In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np 
from torchvision.datasets import MNIST
from scipy.spatial.distance import cdist
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_digits
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd 

In [11]:
with open('./data/Bio_data/Gevers2014_meta.txt', 'r') as file:
    meta = file.read()

# Split the contents into lines
meta_lines = meta.split('\n')

# Remove empty lines and strip whitespace
meta_lines = [line.strip() for line in meta_lines if line.strip()]

# Split each line by tabs to create columns
meta_data = [line.split('\t') for line in meta_lines]

# Create a pandas DataFrame from the list of lists
meta_df = pd.DataFrame(meta_data)

# Count the number of rows and columns
num_rows, num_columns = meta_df.shape

# Print the counts
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Display the first few rows of the DataFrame for inspection
print(meta_df.head())




Number of rows: 1360
Number of columns: 70
                   0                1                     2   \
0           #SampleID  BarcodeSequence  LinkerPrimerSequence   
1  SKBTI.1325.1246591             None   GTGCCAGCMGCCGCGGTAA   
2      121283.1246600             None   GTGCCAGCMGCCGCGGTAA   
3  SKBTI.0870.1246169             None   GTGCCAGCMGCCGCGGTAA   
4  SKBTI.1178.1246304             None   GTGCCAGCMGCCGCGGTAA   

                   3                  4                  5          6   \
0  TARGET_SUBFRAGMENT  ASSIGNED_FROM_GEO  EXPERIMENT_CENTER      TITLE   
1                  V4                  n                 BI  CCFA_RISK   
2                  V4                  n                 BI  CCFA_RISK   
3                  V4                  n                 BI  CCFA_RISK   
4                  V4                  n                 BI  CCFA_RISK   

                      7            8           9   ...                 60  \
0             RUN_PREFIX          AGE  MESALAMINE 

In [12]:
with open('./data/Bio_data/otutable.txt', 'r') as file:
    OTU = file.read()

# Split the contents into lines
OTU_lines = OTU.split('\n')

# Remove empty lines and strip whitespace
OTU_lines = [line.strip() for line in OTU_lines if line.strip()]

# Split each line by tabs to create columns
OTU_data = [line.split('\t') for line in OTU_lines]

# Create a pandas DataFrame from the list of lists
OTU_df = pd.DataFrame(OTU_data)

# Count the number of rows and columns
num_rows, num_columns = OTU_df.shape

# Print the counts
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Display the first few rows of the DataFrame for inspection
print(OTU_df.head())

Number of rows: 944
Number of columns: 1359
                                                0               1     \
0                                            #OTU ID  100052.1246453   
1  NR_112946.1_Bacteroides_vulgatus_strain_JCM_58...            1373   
2  NR_041351.1_Bacteroides_dorei_strain_175_16S_r...              17   
3  NR_041342.1_Parabacteroides_distasonis_strain_...               0   
4  NR_025930.1_Ruminococcus_bromii_strain_ATCC_27...               0   

             2               3               4               5     \
0  100216.1246217  100211.1246832  100083.1246311  100095.1246168   
1            2669               0             311               1   
2            1993               1             165             623   
3             814               0               0              34   
4               0               1               0               0   

             6               7               8               9     ...  \
0  100043.1246460  100212.1247085 

In [14]:
with open('./data/Bio_data/taxatable.txt', 'r') as file:
    TAXA = file.read()

# Split the contents into lines
TAXA_lines = TAXA.split('\n')

# Remove empty lines and strip whitespace
TAXA_lines = [line.strip() for line in TAXA_lines if line.strip()]

# Split each line by tabs to create columns
TAXA_data = [line.split('\t') for line in TAXA_lines]

# Create a pandas DataFrame from the list of lists
TAXA_df = pd.DataFrame(TAXA_data)

# Count the number of rows and columns
num_rows, num_columns = TAXA_df.shape

# Print the counts
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Display the first few rows of the DataFrame for inspection
print(TAXA_df)

Number of rows: 993
Number of columns: 1359
                                                  0               1     \
0                                              #OTU ID  100052.1246453   
1    k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...             826   
2    k__Bacteria;p__Actinobacteria;c__Coriobacterii...             151   
3    k__Bacteria;p__Actinobacteria;c__Actinobacteri...              19   
4                                          k__Bacteria               1   
..                                                 ...             ...   
988  k__Bacteria;p__Synergistetes;c__Synergistia;o_...               0   
989  k__Bacteria;p__Synergistetes;c__Synergistia;o_...               0   
990  k__Bacteria;p__Synergistetes;c__Synergistia;o_...               0   
991  k__Bacteria;p__Tenericutes;c__Mollicutes;o__An...               0   
992  k__Bacteria;p__Verrucomicrobia;c__Verrucomicro...               0   

               2               3               4               5   

In [5]:
meta

,0
0,#SampleID\tBarcodeSequence\tLinkerPrimerSequen...
1,SKBTI.1325.1246591\tNone\tGTGCCAGCMGCCGCGGTAA\...
2,121283.1246600\tNone\tGTGCCAGCMGCCGCGGTAA\tV4\...
3,SKBTI.0870.1246169\tNone\tGTGCCAGCMGCCGCGGTAA\...
4,SKBTI.1178.1246304\tNone\tGTGCCAGCMGCCGCGGTAA\...
...,...
1355,SKBTI.0518.1246864\tNone\tGTGCCAGCMGCCGCGGTAA\...
1356,SKBTI.1317.1247132\tNone\tGTGCCAGCMGCCGCGGTAA\...
1357,SKBTI024.1246988\tNone\tGTGCCAGCMGCCGCGGTAA\tV...
1358,SKBTI.1035.1246669\tNone\tGTGCCAGCMGCCGCGGTAA\...
